In [33]:
import numpy as np
from scipy import linalg

## Linear System of equations using Gauss-Elimination

In [34]:

def backSubst(U,b):
    n = len(U)
    x = np.zeros(n)
    x[n-1] = b[n-1]/U[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i] = (b[i] - sum(U[i,i+1:n]*x[i+1:]))/U[i,i]
    return x

def gaussElim(A,b):
    n = len(A)
    # Make Augmented matrix [A|b]
    Am = np.concatenate((A,b.reshape((n,1))),axis = 1)
    print('\nAugmented Matrix:\n',Am)
    # i iterates over columns
    for i in range(n-1):
        # Do pivoting: if the ith diagonal element is 0 Interchange ith row with the row containing the max absolute value in 
        # ith column below A(i,i)
        if Am[i,i] == 0:
            maxind = np.argmax(abs(Am[i:,i]))
            Am[[i,maxind]] = Am[[maxind,i]]
            print('\nR{} <--> R{}:\n{}'.format(i,maxind,Am))
        # Make ith diagonal equal to 1
        if Am[i,i] != 1.0:
            print('\nR{} --> R{}/{}:'.format(i,i,Am[i,i]))
            Am[i] = Am[i]/Am[i,i]
            print(Am)
        # Make all elements below the ith diagonal in ith column to zero
        for j in range(i+1,n):
            if Am[j,i] != 0:
                print('\nR{} --> R{} - R{}*{}:'.format(j,j,i,Am[j,i]))
                Am[j] -= Am[i]*Am[j,i]
                print(Am)
    print('\nR{} --> R{}/{}:'.format(n-1,n-1,Am[n-1,n-1]))
    Am[n-1] = Am[n-1]/Am[n-1,n-1]
    print(Am)
    # Return A,b after gauss elimination
    return Am[0:n,0:n],Am[:,-1] 

A=np.array([[0,3,-4],[2,-3,7],[-1,0,-9]],float)
b=np.array([16,-27,9],float)
print('A = \n',A,'\nb = \n',b)

Agauss,bgauss = gaussElim(A,b)
print('After gauss elimination:')
print('A = \n',Agauss,'\nb = \n',bgauss)

x = backSubst(Agauss,bgauss)
print('Solution of Ax = b:')
print('x = \n',x)

A = 
 [[ 0.  3. -4.]
 [ 2. -3.  7.]
 [-1.  0. -9.]] 
b = 
 [ 16. -27.   9.]

Augmented Matrix:
 [[  0.   3.  -4.  16.]
 [  2.  -3.   7. -27.]
 [ -1.   0.  -9.   9.]]

R0 <--> R1:
[[  2.  -3.   7. -27.]
 [  0.   3.  -4.  16.]
 [ -1.   0.  -9.   9.]]

R0 --> R0/2.0:
[[  1.   -1.5   3.5 -13.5]
 [  0.    3.   -4.   16. ]
 [ -1.    0.   -9.    9. ]]

R2 --> R2 - R0*-1.0:
[[  1.   -1.5   3.5 -13.5]
 [  0.    3.   -4.   16. ]
 [  0.   -1.5  -5.5  -4.5]]

R1 --> R1/3.0:
[[  1.          -1.5          3.5        -13.5       ]
 [  0.           1.          -1.33333333   5.33333333]
 [  0.          -1.5         -5.5         -4.5       ]]

R2 --> R2 - R1*-1.5:
[[  1.          -1.5          3.5        -13.5       ]
 [  0.           1.          -1.33333333   5.33333333]
 [  0.           0.          -7.5          3.5       ]]

R2 --> R2/-7.5:
[[  1.          -1.5          3.5        -13.5       ]
 [  0.           1.          -1.33333333   5.33333333]
 [ -0.          -0.           1.          -0.4666666

In [35]:
x = linalg.solve(A,b)
print(x)

[-4.8         4.71111111 -0.46666667]


## L-U Decomposition using Doolittle's algorithm:

In [36]:
# Note that matrices with 0 diagonal elements cannot be decomposed to LU = A.
# Hence partial pivoting is not applied.
def LUdecompDoolittle(A):
    a = A.copy()
    n = len(a)
    for i in range(n-1):
        a[i+1:,i] /= a[i,i]
        for j in range(i+1,n):
            a[j,i+1:] -= a[j,i]*a[i,i+1:]
    
    L = np.eye(n,n);U = np.zeros((n,n))
    for i in range(n):
        U[0:i+1,i] = a[0:i+1,i]
        if i!=n-1:
            L[i+1:,i] = a[i+1:,i]
    return L,U

A=(np.linspace(1.0,100.0,25)**0.5).reshape((5,5))
L,U = LUdecompDoolittle(A)
print('A =\n',A)
print('L =\n',L,'\nU =\n',U)
print('L*U =\n',np.dot(L,U))

A =
 [[ 1.          2.26384628  3.04138127  3.65718471  4.18330013]
 [ 4.65026881  5.07444578  5.46580278  5.83095189  6.17454452]
 [ 6.5         6.80991924  7.1063352   7.39087275  7.66485486]
 [ 7.92937576  8.18535277  8.4335639   8.67467579  8.90926484]
 [ 9.13783344  9.36082261  9.57862203  9.79157801 10.        ]]
L =
 [[1.         0.         0.         0.         0.        ]
 [4.65026881 1.         0.         0.         0.        ]
 [6.5        1.44966295 1.         0.         0.        ]
 [7.92937576 1.79083975 1.71452058 1.         0.        ]
 [9.13783344 2.07697194 2.284001   2.06867485 1.        ]] 
U =
 [[ 1.00000000e+00  2.26384628e+00  3.04138127e+00  3.65718471e+00
   4.18330013e+00]
 [ 0.00000000e+00 -5.45304798e+00 -8.67743766e+00 -1.11759401e+01
  -1.32789256e+01]
 [ 0.00000000e+00  0.00000000e+00 -8.32831693e-02 -1.79481609e-01
  -2.76629564e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.47334053e-03
  -6.97894179e-03]
 [ 0.00000000e+00  0.00000000e+00  0.

In [37]:
def LUsolve(L,U,b):
    n = len(b)
    x = np.zeros(n);y = np.zeros(n)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i,0:i],y[0:i])
    x[n-1] = y[n-1]/U[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i] = (y[i] - np.dot(U[i,i+1:],x[i+1:]))/U[i,i]
    return x

b=np.linspace(100.0,200.0,len(A))
print('x = ',LUsolve(L,U,b))

x =  [ -2365.02210705  24512.3557615  -66839.47554511  69957.82600369
 -25241.20740588]


In [38]:
x = linalg.solve(A,b)
print(x)

[ -2365.02210655  24512.35575675 -66839.47553298  69957.82599163
 -25241.20740171]
